In [1]:
import pandas as pd
from datetime import datetime as dt
from pathlib import Path
path = '/home/jupyter/data_ner_ANER/ANERCorp.xlsx'
xlsx = pd.ExcelFile(path)
df = pd.read_excel(xlsx, header=None)
df = df.drop(2, axis=1)
df = df.rename(columns={0: 'text', 1:'label'})

In [90]:
df_corrected = df.copy()
tag_dict = {'O','B-PER','I-PER','B-LOC','I-LOC','B-ORG','I-ORG','B-MISC','I-MISC'}
delete_line = []
for index, row in df_corrected.iterrows():
    if row['label'] in {'B-PERS', 'B-PERs', 'B-ERS'}:
        row['label'] = 'B-PER' 
    elif row['label'] in {'I-PERS', 'IPERS'}:
        row['label'] = 'I-PER'
    elif row['label'] in {'O"', 'o'}:
        row['label'] = 'O'
    elif row['label'] in {'I-PRG'}:
        row['label'] = 'I-ORG'
    elif row['label'] in {'B-PRG', 'B-OEG','B-ORF'}:
        row['label'] = 'B-ORG'
    elif row['label'] in {'B-LOCI-PERS'}:
        row['label'] = 'B-LOC'
    elif row['label'] in {'b-misc', 'B-MSIC'}:
        row['label'] = 'B-MISC'
    elif row['label'] in {'i-misc'}:
        row['label'] = 'I-MISC'       
    elif row['label'] in {'00:00:00"'}:
        row['label'] = 'O'
    elif row['label'] in tag_dict:
        continue
        
        
    if row['text'] in {' O', 'O"', 'O'}:
        delete_line.append(index)
        
    if row['text'] in {str(b'\xd9\x81\xd9\x8a\xd8\xb1\xd8\xb3\xd9\x84\xd8\xa7\xd9\x8a\xd9\x86', 'utf-8'), 
                       str(b'\xd8\xa7\xd9\x84\xd8\xa5\xd8\xb5\xd8\xa7\xd8\xa8\xd8\xa9', 'utf-8'),
                       str(b'\xe2\x80\x8f', 'utf-8')}:
        print('strange text')
        print(row)
        delete_line.append(index)
        
        
    try:
        if re.search(r"\s", row['text']) and (row['text'] != ' O'):
            print("several words")
            print(row['text'])
            delete_line.append(index)
    except:
        print("could not use re")
        delete_line.append(index)
        print(row)


several words
. O
strange text
text     فيرسلاين
label       B-PER
Name: 83703, dtype: object
could not use re
text        16
label    I-PER
Name: 128136, dtype: object
could not use re
text        16
label    I-PER
Name: 128367, dtype: object


In [91]:
df_new = df_corrected.drop(delete_line)
fname = '/home/jupyter/data_ner_ANER/normalized_file.txt'
df_new.to_csv(fname, sep='\t', index=False, encoding='utf-8', header=False)

# insert a newline when there is a .
fname_input = fname
fname_output = '/home/jupyter/data_ner_ANER/normalized_file_separate_sentence.txt' # should you actually get rid of

f_in = open(fname_input, 'r', encoding='utf-8')
f_out = open(fname_output, 'w', encoding='utf-8')
for r in f_in.readlines():
    f_out.write(r)
    token = r.split('\t')

    if token[0] == '.':
        f_out.write('\n')
f_in.close()
f_out.close()

# split the data into train, dev, and text. 90, 10.
fname_full = fname_output
fname_train = '/home/jupyter/data_ner_ANER/train.txt'
fname_dev = '/home/jupyter/data_ner_ANER/dev.txt'
f_in = open(fname_full, 'r', encoding='utf-8')
f_out_train = open(fname_train, 'w', encoding='utf-8')
f_out_dev = open(fname_dev, 'w', encoding='utf-8')

count = 0
for r in f_in.readlines():
    count += 1
num_train = int(count * 0.9)
print(count, num_train)
f_in.close()

f_in = open(fname_full, 'r', encoding='utf-8')

for ii, r in enumerate(f_in.readlines()):
    if ii < num_train:
        f_out_train.write(r)
    else:
        f_out_dev.write(r)
f_in.close()
f_out_train.close()
f_out_dev.close()

153437 138093
